# Visualization Examples
- [IpyLeaflet usage examples](https://towardsdatascience.com/ipyleaflet-interactive-mapping-in-jupyter-notebook-994f19611e79)
- [Awesome icons](https://fontawesome.com/v4.7.0/icon/medkit)

In [23]:
import csv
import os
import datetime
import geojson
import json

import geopandas as gpd
from ipyleaflet import (
    Map, Marker, MarkerCluster, TileLayer, 
    ImageOverlay, AwesomeIcon, 
    LayersControl, ScaleControl, ZoomControl, FullScreenControl,
    MeasureControl, SplitMapControl,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, 
    GeoData, GeoJSON, 
    LayerGroup,
    basemaps, basemap_to_tiles
)
from ipywidgets import HTML
import psycopg2
from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.wkb import loads
from shapely.wkt import dumps, loads

In [ ]:
%%bash

# head -n 5 data/patient_branch_zip.csv
# pip3 list

In [2]:
# Postgis
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]
branch_file = "data/patient_branch_zip.csv"

## Create Map
- [Basemaps](https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html)

In [ ]:
# center (lat, lon)
centroid = [35.0984955, -85.3791176]

# mapper = Map(center=centroid, zoom=5, basemap=basemaps.CartoDB.DarkMatter)
# mapper = Map(center=latlon, zoom=zoom, basemap=basemaps.Esri.WorldStreetMap)
# mapper = Map(center=latlon, zoom=zoom, basemap=basemaps.OpenStreetMap.HOT)
# mapper = Map(center=latlon, zoom=zoom, basemap=basemaps.CartoDB.Positron)
# mapper = Map(center=latlon, zoom=zoom, basemap=basemaps.OpenStreetMap.Mapnik)
mapper = Map(center=latlon, zoom=zoom, basemap=basemaps.Stamen.Toner)
# mapper = Map(center=latlon, zoom=zoom, basemap=basemaps.NASAGIBS.ViirsEarthAtNight2012)
mapper

In [ ]:
mapper.layers

## Create Map from SQL

### Add branches
- Also, saves to shapefile from postgreSQL

In [32]:
%%time
branch_dml = """
    SELECT  DISTINCT
            branch_key,
            branch_latitude,
            branch_longitude,
            branch_location
            --ST_AsGeoJSON(branch_location) AS branch_json
    FROM    public.patient_branch
"""

with psycopg2.connect(host=pg_host, 
                      database=pg_db,
                      user=pg_user, 
                      password=pg_pwd) as c:
    patient_df = gpd.GeoDataFrame.from_postgis(sql=branch_dml, con=c,
                                               geom_col="branch_location")
#     patient_df.to_file('data/patient_branch.shp')
#     patient_3857_df = patient_df.copy().to_crs(epsg=3857)  # web mercator
#     patient_3857_df.to_file('data/patient_branch.geojson', driver='GeoJSON')
    patient_df.to_csv('data/patient_branch.csv', index=False)
    
with psycopg2.connect(host=pg_host, 
                      database=pg_db,
                      user=pg_user, 
                      password=pg_pwd) as c:
    try:
#         mapper.clear_layers()
#         mapper.add_layer(basemaps.OpenStreetMap.Mapnik)
        
        # Execute DML
        cursor = c.cursor()
        cursor.execute(branch_dml)
        
        centroid = [35.0984955, -85.3791176]
        hospital_icon = AwesomeIcon(
            name='fa-medkit',
            marker_color='red',
            icon_color='white',
            spin=False
        )
        mapper = Map(center=centroid, zoom=6, zoom_control=False)
        
        left_map = basemap_to_tiles(basemaps.Stamen.Toner)
        right_map = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)

        split_control = SplitMapControl(left_layer=left_map, right_layer=right_map)
        mapper.add_control(split_control)
        
        branch_group = LayerGroup(name="Branches")
        branch_radius_group = LayerGroup(name="Branch Radius")
        
        shapes = 0
        for rec in cursor:
            shapes += 1
            
            branch_loc = (rec[1], rec[2])
            marker = Marker(location=branch_loc, opacity=0.5, 
#                             rise_on_hover=True, rise_offset=250,
                            icon=hospital_icon)
            branch_group.add_layer(marker);
            
            branch_info = HTML()
            branch_info.value = f"Branch Key: <b>{rec[0]}</b>"
#             branch_info.placeholder = "Some HTML"
#             branch_info.description = "Some HTML"
            marker.popup = branch_info
        
            branch_r = Circle()
#             branch_r = CircleMarker()
            branch_r.location = branch_loc
#             branch_r.radius = 10  # CircleMarker pixels
            branch_r.radius = 6000  # Circle meters
            branch_r.fill = True
            branch_r.fill_color = "yellow"
            branch_r.fill_opacity = 0.5
            branch_r.stroke = True
            branch_r.color = "#000000"  # stroke color, also hex #0033FF
            branch_r.opacity = 0.7  # stroke
            branch_r.weight = 1  # stroke
            
            branch_radius_group.add_layer(branch_r)

        mapper.add_layer(branch_group)
        mapper.add_layer(branch_radius_group)
        
        mapper.add_control(ZoomControl(position='topright'))
        mapper.add_control(LayersControl())
        mapper.add_control(ScaleControl(position='bottomleft', metric=False))
        mapper.add_control(FullScreenControl())
        
        measure = MeasureControl(
            position='bottomleft',
            active_color = 'orange',
            primary_length_unit = 'miles'
        )
        mapper.add_control(measure)

        measure.completed_color = 'red'

#         measure.add_length_unit('yards', 1.09361, 4)
#         measure.secondary_length_unit = 'yards'
#         measure.add_area_unit('sqyards', 1.19599, 4)
#         measure.secondary_area_unit = 'sqyards'
        
        print(f"Added {shapes} branches")
        
    except Exception as e:
        raise e

display(mapper)

Added 55 branches


Map(center=[35.0984955, -85.3791176], controls=(AttributionControl(options=['position', 'prefix'], position='b…

CPU times: user 1.43 s, sys: 55.7 ms, total: 1.49 s
Wall time: 1.55 s


### Add Patients
- Loads from SQL to GeoDataFrame

In [25]:
%%time
patient_dml = """
    select  patient_latitude
            , patient_longitude
            , patient_location
            , MIN(patient_key) AS patient_key
            , COUNT(*) OVER () AS total_patients
    from    public.patient_branch
    GROUP BY patient_latitude, patient_longitude, patient_location
"""

with psycopg2.connect(host=pg_host, 
                      database=pg_db,
                      user=pg_user, 
                      password=pg_pwd) as c:
    patient_df = gpd.GeoDataFrame.from_postgis(sql=patient_dml, con=c,
                                               geom_col="patient_location")

with psycopg2.connect(host=pg_host, 
                      database=pg_db,
                      user=pg_user, 
                      password=pg_pwd) as c:
    try:
#         mapper.clear_layers()
#         mapper.add_layer(basemaps.OpenStreetMap.Mapnik)

        patient_icon = AwesomeIcon(
            name='fa-user-o',
            marker_color='blue',
            icon_color='white',
            spin=False
        )

        patient_loc = [Marker(location=(geo.x, geo.y), 
                              icon=patient_icon, 
#                               rise_on_hover=True, rise_offset=250
                             ) 
                       for geo in patient_df.geometry
                      ]
        mapper.add_layer(MarkerCluster(markers=patient_loc, name="Patients"))
        print(f"Added {patient_df.shape[0]} patients")

    except Exception as e:
        raise e

display(mapper)

Added 646 patients


Map(bottom=52072.0, center=[35.10193405724608, -85.374755859375], controls=(AttributionControl(options=['posit…

CPU times: user 3.21 s, sys: 113 ms, total: 3.32 s
Wall time: 3.47 s


In [226]:
len(mapper.layers)

4